<div><center>
<table><tr>
<td>
<img src="images/Embraer_logo.png" alt="Empresa Brasileira de Aeronáutica (Embraer)" style="width: 300px;"/>
    </td>
<td>
<img src="images/ITA_logo.png" alt="Instituto Tecnológico da Aeronáutica (ITA)" style="width: 300px;"/>
    </td>
    </tr></table
</center></div>

In [ ]:
import model_reference as mf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings

plt.style.use('seaborn')
warnings.filterwarnings('ignore')


# EMBRAER | ITA

## ENGENHARIA AERONÁUTICA E MECÂNICA - PEE 30

### AC-701 : SISTEMAS PROPULSIVOS - PROF. LACAVA

### Autores:

* Camilla Santos
* Eloi Antonio
* Gabriel Henrique
* Jonas Degrave
* Lucas Schroeder
* Matheus Monteverde
* Orlando Gabriel
* Rafael Bayão

---

## Exercício 1: Montagem do Simulador

* Matheus Monteverde
* Rafael Bayão

### Estratégia para criação do simulador

O simulador foi montado a partir de uma arquitetura de Programação Orientada a Objetos. Dessa forma, foram modelados processos que são herdados pelos componentes e esses são utilizados pelos motores a medida da necessidade. Os a lista de objetos e suas funções podem ser visualizados abaixo:

Processos térmicos: cada processo armazena as propriedades adequadas para formulação de suas transformações.

* ThermalProcess: Classe principal - implementa métodos genéricos isentrópicos e é herdada pelas demais.
* StaticThermalProcess: implementa métodos para processos com entrada e saída com velocidades desprezíveis para o gás.
* SpeedInThermalProcess: implementa métodos para processos com entrada a velocidade considerável e saída a velocidade desprezível.
* SpeedOutThermalProcess: implementa métodos para processos com entrada a velocidade desprezível e saída a velocidade consideravel.

<center><img src="images/questao1/thermal-processes.png" alt="Thermal Process"/></center>

Componentes: cada componente herda um processo térmico adequado ao seu funcionamento implementa as equações necessárias. Os valores de entrada são armazenados e os de saída calculados ao serem invocados. Por fim, implementam métodos para variar a rotação e resumir todos os parametros inclusos.

* Diffuser_Adiab
* Nozzle_Adiab
* Compressor
* Fan
* Turbine
* FanTurbine
* FreeTurbine
* CombustionChamber

Motores: cada motor instancia sequencialmente os componentes necessários para o funcionamento e implementa métodos para ajustar globalmente a rotação e fornecer todas as propriedades de entrada e saída dos componentes e do próprio motor.

* TurboJet
* TurboFan
* TurboProp

<div><center>
<table><tr>
<td>
<img src="images/questao1/turbojet-schema.png" alt="turbojet-schema" style="height: 400px;"/>
    </td>
<td>
<img src="images/questao1/turbofan-schema.png" alt="turbofan-schema" style="height: 400px;"/>
    </td>
    <td>
<img src="images/questao1/turboprop-schema.png" alt="turboprop-schema" style="height: 400px;"/>
    </td>
    </tr></table
</center></div>

---

### Exemplo 2.1: TurboJet

#### Informações do exercício:

Calcular o empuxo específico e consumo específico (TSFC) para um motor turbojato operando a Mach 0,85 e a 12.200 m (Pa = 18,75kPa e Ta = 216,7 K). Usar os dados abaixo para eficiências e razões de calor específico.

| Componente | Eficiência Adiabática | $\gamma$ |
| :--------- |:---------------------:| -------- |
| difusor    | 0,97                  | 1,40     |
| compressor | 0,85                  | 1,37     |
| combustão  | 1,00                  | 1,35     |
| turbina    | 0.90                  | 1,33     |
| bocal      | 0,98                  | 1,36     |

Dados operacionais


| Dados operacionais:                         | Valor             |
| ------------------------------------------- | ----------------- |
| Razão de Pressão no compressor              | 30                |
| Temperatura na sáida da câmara de combustão | 1600 K            |
| Poder Calorífico Inferior do Combustível    | 45000 kJ/kg       |
| R Médio                                     | 288,3 m^2/(s^2.K) |
| Cp no combustor                             | 1,11 kJ/(kg.K)    |



In [ ]:
values_ex2_1 = {
    'ta': 216.7,         # Temperatura ambiente
    'pa': 18.750,        # Pressão atmosférica
    'gamma_d': 1.4,      # gamma no difusor
    'gamma_c': 1.37,     # gamma no compressor
    'gamma_b': 1.35,     # gamma na combustão
    'gamma_t': 1.33,     # gamma na turbina
    'gamma_n': 1.36,     # gamma no bocal
    'n_d': 0.97,         # eficiência no difusor
    'n_c': 0.85,         # eficiência no compressor
    'n_b': 1,            # eficiência na combustão
    'n_t': 0.9,          # eficiência na turbina
    'n_n': 0.98,         # eficiência no bocal
    'prc': 30,           # Taxa de compressão - compressor
    'pc_fuel': 45000,    # Poder Calorífico do combustível
    'cp_fuel': 1.110,    # Capacidade calorífica do ar no combustor
    'r': 288.3,          # R médio do ar
    't04': 1600,         # Temperatura na saída da camera de combustão
    'mach': 0.85         # mach de voo
}

turbojet = mf.TurboJet(values_ex2_1)    # Instanciando o motor
targets = ['t02', 'p02', 't03', 'p03', 'f', 't05', 'p05', 'u_s', 'TSFC',
           'u_i', 'specific_thrust']    # Definindo variáveis de interesse
result_2_1 = (
    turbojet.sumarise().
    append(turbojet.sumarise_results()).
    loc[targets]
)
clean_names = ['Temperatura na Saída Difusor', 'Pressão na Saída Difusor',
               'Temperatura na Saída Compressor', 'Pressão na Saída Compressor',
               'Razão combustível/ar', 'Temperatura na Saída Turbina',
               'Pressão na Saída Turbina', 'Velocidade de Saída do ar',
               'Consumo específico', 'Velocidade de voo', 'Empuxo específico']
result_2_1.columns = ['Valores']
result_2_1.rename(index=dict(zip(targets, clean_names)))


#### Comparação de resultados

| Parâmetro                                      | Unidade   | Modelo   | Referência do material | $\Delta$ em % |
| ---------------------------------------------- | --------- | -------- | ---------------------- | ------------- |
| Temp. total no final da entrada de ar          | K         | 248.013  | 248,013                | 0             |
| Press. total no final da entrada de ar         | kPa       | 296.7483 | 29,6748                | 0             |
| Press. Total na saída do Compressor            | kPa       | 890,245  | 890,245                | 0             |
| Temp. total na saída do Compressor             | K         | 687,349  | 687,349                | 0             |
| Razão Combustível/Ar                           | -         | 0,02344  | 0,02344                | 0             |
| Temp. total na saída da turbina do compressor  | K         | 1160,66  | 1160,66                | 0             |
| Press. total na saída da turbina do compressor | kPa       | 205,316  | 205,316                | 0             |
| Velocidade de saída                            | m/s       | 1078,3   | 1078,3                 | 0             |
| Consumo Específico                             | kg/(kN.s) | 0,0275   | 0,0275                 | 0             |
| Velocidade de vôo                              | m/s       | 251,382  | 251,382                | 0             |
| Empuxo Específico                              | kN.s/kg   | 0,85219  | 0,85219                | 0             |


---

### Exemplo 2.2: TurboFan

#### Informações do exercício

Calcular o empuxo específico e consumo específico (TSFC) para um motor turbofan operando a Mach 0,85 e a 12.200 m (Pa = 18,75 e Ta = 216,7 K). Usar os dados abaixo para eficiências e razões de calor específico. Comparar com exemplo anterior para o motor turbojato.

| Componente             | Eficiências Adiabática | $\gamma$ |
| ---------------------- |:-----------:| -------- |
| Entrada de ar          | 0,97        | 1,40     |
| Fan                    | 0,85        | 1,40     |
| Compressor             | 0,85        | 1,37     |
| Câmara de combustão    | 1,00        | 1,35     |
| Turbina do compressor  | 0.90        | 1,33     |
| Turbina do fan         | 0.90        | 1,33     |
| Bocal de gases quentes | 0,98        | 1,36     |
| Bocal do fan           | 0,98        | 1,40     |

Dados operacionais:

| Parâmetro                                   | Valores         |
| ------------------------------------------- | --------------- |
| Temperatura na sáida da câmara de combustão | 1600 K          |
| Razão de Pressão no compressor              | 20              |
| Razão de Pressão no Fan                     | 1.5             |
| Razão de Passagem                           | 5.0             |
| Poder Calorífico Inferior do Combustível    | 45000 kJ/kg     |
| R Médio                                     | 288,3 m2/(s2.K) |
| Cp no combustor                             | 1,11 kJ/(kg.K)  |



In [ ]:
values_ex2_2 = {
    'ta': 216.7,         # Temperatura ambiente
    'pa': 18.750,        # Pressão atmosférica
    'n_d': 0.97,         # eficiência do difusor
    'n_c': 0.85,         # eficiência do compressor
    'n_f': 0.85,         # eficiência do fan
    'n_b': 1,            # eficiência na combustão
    'n_t': 0.9,          # eficiência da turbina
    'n_tf': 0.9,         # eficiência da turbina do fan
    'n_n': 0.98,         # eficiência do bocal
    'n_nf': 0.98,        # eficiência do bocal do fan
    'gamma_d': 1.4,      # gamma no difusor
    'gamma_f': 1.4,      # gamma no fan
    'gamma_c': 1.37,     # gamma no compressor
    'gamma_b': 1.35,     # gamma na combustão
    'gamma_t': 1.33,     # gamma na turbina
    'gamma_tf': 1.33,    # gamma na turbina do fan
    'gamma_n': 1.36,     # gamma no bocal
    'gamma_nf': 1.4,     # gamma no bocal do fan
    'prc': 20,           # Taxa de compressão - compressor
    'prf': 1.5,          # Taxa de compressão - fan
    'bypass_ratio': 5,   # razão de passagem
    'pc_fuel': 45000,    # Poder Calorífico do combustível
    'cp_fuel': 1.110,    # Capacidade calorífica do ar no combustor
    'r': 288.3,          # R médio do ar
    't04': 1600,         # Temperatura na saída da camera de combustão
    'mach': 0.85         # mach de voo
}

turbofan = mf.TurboFan(values_ex2_2)    # Instanciando o motor
target = [                              # Variáveis de interesse
    't02', 'p02',
    't08', 'p08',
    't03', 'p03',
    'f',
    'tet', 'pet',
    't05', 'p05',
    'u_s', 'u_sf', 'u_i',
    'specific_thrust', 'TSFC'
]
result_2_2 = (                          # Selecionando valores
    turbofan.sumarise().
    append(turbofan.sumarise_results()).
    loc[target]
)
result_2_2.columns = ['turbofan']
result_2_2['turbojet'] = result_2_1     # Juntando resultados

result_2_2


#### Comparação de resultados do modelo do Turbo Fan com o material

| Parâmetro                                      | Unidade   | Modelo  | Referência do material | $\Delta$ em % |
| ---------------------------------------------- | --------- | ------- | ---------------------- | ------------- |
| Temp. total no final da entrada de ar          | K         | 248,013 | 248,013                | 0             |
| Press. total no final da entrada de ar         | kPa       | 29,6748 | 29,6748                | 0             |
| Press. Total na saída do Fan                   | kPa       | 29,6748 | 29,6748                | 0             |
| Temp. total na saída do Fan                    | K         | 283,851 | 283,851                | 0             |
| Press. Total na saída do Compressor            | kPa       | 890,245 | 890,245                | 0             |
| Temp. total na saída do Compressor             | K         | 699,879 | 699,879                | 0             |
| Razão Combustível/Ar                           | -         | 0,02312 | 0,02312                | 0             |
| Temp. total na saída da turbina do Compressor  | K         | 1183,97 | 1183,97                | 0             |
| Press. total na saída da turbina do Compressor | kPa       | 225,281 | 225,281                | 0             |
| Temp. total na saída da turbina do Fan         | K         | 968,945 | 968,945                | 0             |
| Press. total na saída da turbina do Fan        | kPa       | 90,8277 | 90,8277                | 0             |
| Velocidade de saída - Bocal do fan             | m/s       | 350,532 | 350,532                | 0             |
| Velocidade de saída - Bocal de gases quentes   | m/s       | 840,335 | 840,335                | 0             |
| Velocidade de vôo                              | m/s       | 251,382 | 251,382                | 0             |
| Empuxo Específico                              | kN.s/kg   | 1,10413 | 1,10413                | 0             |
| Consumo Específico                             | kg/(kN.s) | 0,02094 | 0,02094                | 0             |



---

### Exemplo 2.3: TurboFan - variação da rotação a partir do ponto de projeto

#### Informações do exercício

Condição de Projeto do Motor - Máxima Rotação - Nível do mar - 288,15K

| Parâmetros                                  | Valores  |
| ------------------------------------------- | -------- |
| Vazão de ar                                 | 756 kg/s |
| Tenoeratura na saída da câmara de combustão | 1550 K   |
| Razão de Pressão no compressor              | 17.2     |
| Razão de Pressão no Booster                 | 1.00     |
| Razão de Pressão no Fan                     | 1.69     |
| Razão de Passagem                           | 4.69     |
| % de perda de pressão taotal na câmara      | 5 %      |

Condições de Operação:

| Condições de operação em voo      | Valores    |
| --------------------------------- | ---------- |
| Mach de voo                       | 0          |
| Pressão atmosférica               | 101.63 kPa |
| Temperatura atmosférica           | 290 K      |
| % da rotação máxima no compressor | 1.00       |
| % da rotação máxima no fan        | 1.00       |



Eficiências e valores de $\gamma$:

| Componente             | Eficiências Adiabática | $\gamma$ |
| ---------------------- |:----------------------:| -------- |
| Entrada de ar          | 0,97                   | 1,40     |
| Fan                    | 0,93                   | 1,40     |
| Compressor             | 0,90                   | 1,37     |
| Câmara de combustão    | 0,9995                 | 1,35     |
| Turbina do compressor  | 0.95                   | 1,33     |
| Turbina do fan         | 0.932                  | 1,33     |
| Bocal de gases quentes | 0,98                   | 1,36     |
| Bocal do fan           | 0,98                   | 1,40     |


Dados termodinâmicos:

| Dados termodinâmicos                     | Valores         |
| ---------------------------------------- | --------------- |
| Poder Calorífico Inferior do Combustível | 45000 kJ/kg     |
| R Médio                                  | 288,3 m2/(s2.K) |
| Cp no combustor                          | 1,11 kJ/(kg.K)  |



#### Solução

In [ ]:
values_ex2_3 = {
    'ta': 290,               # Temperatura ambiente
    'pa': 101.630,           # Pressão atmosférica
    'n_d': 0.97,             # eficiência do difusor
    'n_f': 0.93,             # eficiência do fan
    'n_c': 0.90,             # eficiência do compressor
    'n_b': 0.9995,           # eficiência na combustão
    'n_t': 0.95,             # eficiência da turbina
    'n_tf': 0.932,           # eficiência da turbina do fan
    'n_n': 0.98,             # eficiência do bocal
    'n_nf': 0.98,            # eficiência do bocal do fan
    'gamma_d': 1.4,          # gamma no difusor
    'gamma_f': 1.4,          # gamma no fan
    'gamma_c': 1.37,         # gamma no compressor
    'gamma_b': 1.35,         # gamma na combustão
    'gamma_t': 1.33,         # gamma na turbina
    'gamma_tf': 1.33,        # gamma na turbina do fan
    'gamma_n': 1.36,         # gamma no bocal
    'gamma_nf': 1.4,         # gamma no bocal do fan
    'prc': 17.2,             # Taxa de compressão - compressor
    'prf': 1.69,             # Taxa de compressão - fan
    'bypass_ratio': 4.749,   # razão de passagem
    'pc_fuel': 45000,        # Poder Calorífico do combustível
    'cp_fuel': 1.110,        # Capacidade calorífica do ar no combustor
    'r': 288.3,              # R médio do ar
    't04': 1550,             # Temperatura na saída da camera de combustão
    'mach': 0,               # mach de voo
    'mass_flow': 756,        # vazão mássica
    'pressure_loss': 0.05,   # perda de pressão na câmara de combustão
}

turbofan23 = mf.TurboFan(values_ex2_3)      # Instanciando motor

summarise_target = [                        # Variáveis de interesse
    'n1',
    'thrust_total',
    'TSFC',
    'prf', 'prc',
    't04',
    'hot_mass_flow', 'bypass_ratio']


results = (                                 # Iniciando tabela de resultados
    turbofan23.sumarise().
    append(turbofan23.sumarise_results()).
    loc[summarise_target]
)

for i in np.arange(0.9, 0.4, -0.1):           # Iterando sob diferentes rotações N2
    turbofan23.set_n2(i)
    results[i] = (
        turbofan23.sumarise().
        append(turbofan23.sumarise_results()).
        loc[summarise_target]
    )

results.loc['total_fuel_comsuntion'] = results.loc['TSFC'] * \
    results.loc['thrust_total']
results.loc['pr_global'] = results.loc['prc'] * results.loc['prf']

ordered_target = ['n1', 'thrust_total', 'TSFC', 'prf', 'prc', 'pr_global',
                  't04', 'hot_mass_flow', 'total_fuel_comsuntion', 'bypass_ratio']
clean_names = ['Rotação do Fan N1', 'Empuxo Total', 'TSFC', 'Prf', 'Prc', 'Taxa de comp. Global',
               'Temp. Saída do Combustor', 'Vazão mássica de ar quente', 'Consumo', 'Razão de Passagem']
results = results.loc[ordered_target]


In [ ]:
print(results.rename(index=dict(zip(ordered_target, clean_names))).to_markdown())


In [ ]:
# Lista de nomes para composição dos gráficos
y_labels = ['Rotação N1 [%P.P]', 'Empuxo [kN]',
            'TSFC [kg/(kN.s)]', '$P_{rf}$', '$P_{rc}$', '$P_r$ Global', 'T04 [K]', '$\dot{m}_{a_H}$ [kg/s]', 'Consumo [kg/s]', 'B']
titles = ['N1 x N2', 'Empuxo x N2', 'Consumo Específico x N2',
          '$P_{rf}$ x N1', '$P_{rc}$ x N2', '$P_r$ Global x N2', 'T04 x N2', '$\dot{m}_{a_H}$ x N2', 'Consumo de Combustível x N2', 'Razão de Passagem x N1']
yticks = [np.arange(0, 1.3, 0.2), np.arange(0, 301, 50), np.arange(0, 0.017, 0.002), np.arange(0, 1.9, 0.2), np.arange(
    0, 21, 2), np.arange(0, 36, 5), np.arange(0, 1801, 200), np.arange(0, 161, 20), np.arange(0, 3.1, 0.5), np.arange(2, 9, 1)]


In [ ]:
# Plotagem de resultados conforme material
fontsize = 14
cols = 2
rows = 5
fig, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(16, 20))
fig.subplots_adjust(hspace=.5)
aux = 0
for i in range(0, rows):
    for j in range(0, cols):
        plt.sca(ax[i, j])
        if results.index[aux] == 'prf':
            plt.plot(results.loc['n1'], results.iloc[aux], marker='o')
            plt.axis([0.2, 1.01, 0, max(results.iloc[aux])*1.2])
            plt.xlabel('Rotação N1', fontsize=fontsize)

        elif results.index[aux] == 'bypass_ratio':
            plt.plot(results.loc['n1'], results.iloc[aux], marker='o')
            plt.axis([0.2, 1.01, 2, max(results.iloc[aux])*1.2])
            plt.xlabel('Rotação N1', fontsize=fontsize)
        else:
            plt.plot(results.iloc[aux], marker='o')
            plt.axis([0.4, 1.01, 0, max(results.iloc[aux])*1.2])
            plt.xlabel('Rotação N2', fontsize=fontsize)
        plt.ylabel(y_labels[aux], fontsize=fontsize)
        plt.title(titles[aux], fontsize=fontsize)
        plt.yticks(yticks[aux])
        aux += 1


| Parâmetro                  |        1.0 |        0.9 |       0.8 |        0.7 |       0.6 |       0.5 |
| :------------------------- | ---------: | ---------: | --------: | ---------: | --------: | --------: |
| Rotação do Fan N1          |          1 |    0.87016 |    0.7285 |    0.58684 |   0.44518 |   0.30352 |
| Empuxo Total               |    253.464 |    168.517 |   90.0758 |    48.2752 |   22.8478 |    12.024 |
| TSFC                       | 0.00996902 | 0.00911273 | 0.0092772 | 0.00898977 | 0.0100623 | 0.0131675 |
| Prf                        |       1.69 |    1.56341 |   1.38538 |    1.24073 |   1.12948 |   1.05162 |
| Prc                        |       17.2 |    14.3017 |   10.9042 |    7.59124 |   4.98958 |   3.72595 |
| Taxa de comp. Global       |     29.068 |    22.3593 |   15.1064 |    9.41871 |   5.63565 |   3.91828 |
| Temp. Saída do Combustor   |    1529.52 |    1345.31 |   1166.82 |    1013.36 |   884.936 |   781.537 |
| Vazão mássica de ar quente |    131.056 |    95.2792 |   63.0639 |    38.8194 |   23.4889 |   18.8928 |
| Consumo                    |    2.52679 |    1.53565 |  0.835651 |   0.433983 |  0.229901 |  0.158326 |
| Razão de Passagem          |      4.749 |    5.38464 |   6.01871 |    6.49411 |   6.81085 |   6.96894 |

<center><img src="images/questao1/ex2_3.jpg" alt="turboprop-schema" style="width:1000px"/></center>

#### Comparação de resultados

Para facilitar a comparação de resultados optou-se por gerar respostas para os mesmos pontos que rotação que os apresentados no material de referência. Além disso, foi utilizado os mesmos valores nos eixos e grade. Por não ter os valores exatos no material, apenas uma análise comparativa visual foi possível.

Nesse sentido, verificou-se que todos os gráficos apresentaram a mesma tendência de aumento ou redução do valor de interesse, na maioria os valores são aparentemente os mesmos para o modelo e o material de referência. Apesar disso, houve variações perceptíveis para os resultados de consumo específico de combustível e consumo de combustível, esses valores variam principalmente para rotações menores de 80% em que começam a ter problemas de pressão no bocal de saída dos gases quentes. Nesse ponto a pressão de entrada do bocal se torna menor que a atmosférica e foi optado por definir a velocidade de saída como 0 para poder calcular o restante dos valores.

---

### Exemplo 7.1: Turboélice - Calibrando o Modelo

#### Descrição do Exercício

Tomando os dados do motor PW127 ao nível do mar e estacionário: potência de eixo da turbina livre 2457 kW, vazão de ar no
compressor de 8,49 kg/s, razão de pressão do compressor 15,77 e temperatura de saída da câmara de combustão de 1600K. Fazer uma calibração do motor na simulação de desempenho para que se obtenha uma potência de eixo próxima à informada pelo fabricante.

Parâmetros de funcionamento do motor:

| Condição de funcionamento de referência     | Valores   |
| ------------------------------------------- | --------- |
| Vazão de massa                              | 8,49 kg/s |
| Temperatura de saída na câmara de combustão | 1600 K    |
| Razão de Pressão no Compressor              | 15.77     |


Condição de funcionamento:

| Condição de funcionamento         | Valores    |
| --------------------------------- | ---------- |
| Flight Mach                       | 0          |
| Pressão atmosférica               | 101,30 kPa |
| Temperatura atmosférica           | 288.15 K   |
| % de rotação máxima do compressor | 1,00       |


Eficiência e dados específicos sobre a relação de calor:

| Componente            | Eficiência Adiabática | $\gamma$ |
| --------------------- |:---------------------:| -------- |
| entrada               | 0,85                  | 1,40     |
| compressor            | 0,75                  | 1,37     |
| combustão             | 1,00                  | 1,35     |
| turbina do compressor | 0,90                  | 1,33     |
| Turbina livre         | 0,90                  | 1,33     |
| bocal de ar quente    | 0,98                  | 1,36     |

Dados termodinâmicos:

| Dados Termodinâmicos                    | Valores           |
| --------------------------------------- | ----------------- |
| PCI                                     | 45000 kJ/kg       |
| Média R                                 | 288,3 m^2/(s^2.K) |
| calor específico na câmara de combustão | 1,11 kJ/(kg.K)    |








#### Parte 1:  Ajuste da razão de pressão
Calibrar o modelo para o relatório do fabricante, variando a razão de pressão na turbina para atingir a potência da turbina especficiada de 2457 kW.

In [ ]:
values = {
    'mass_flow': 8.49,
    't04': 1600,
    'prc': 15.77,
    'u_in': 0,
    'ta': 288.15,
    'pa': 101.30,
    'n_d': 0.85,
    'gamma_d': 1.4,
    'n_c': 0.75,
    'gamma_c': 1.37,
    'n_b': 1,
    'gamma_b': 1.35,
    'n_t': 0.9,
    'gamma_t': 1.33,
    'n_tl': 0.9,
    'gamma_tl': 1.33,
    'n_n': 0.98,
    'gamma_n': 1.36,
    'pc_fuel': 45000,
    'cp_fuel': 1.110,
    'cp_tls': 1.160,
    'cp_tl': 1.160,
    'r': 288.3,
    'pr_tl': 2.87,
    'gearbox_power_ratio': 0.98,
    'propeller_efficiency': 0.85,
}


target = [
    'specific_power_turbine',
    'turbine_power',
    'gearbox_power',
    'specific_thrust',
    'BSFC',
    'EBSFC',
    'TSFC',
    'thrust_hot_air',
    'thrust_propeller',
    'fuel_consumption',
    'thrust_total',
    'mass_flow',
    'aircraft_speed'
]

list_of_turbine_pressure_ratio = np.linspace(1.8, 3.4, 10)
turboprop = mf.TurboProp(values)
performance_results = turboprop.sumarise_results()
performance_results = performance_results.drop(columns=[1])

for turbine_pressure_ratio in list_of_turbine_pressure_ratio:
    values["pr_tl"] = turbine_pressure_ratio
    turboprop = mf.TurboProp(values)
    performance_result = turboprop.sumarise_results()
    performance_results[turbine_pressure_ratio] = performance_result


fig1 = plt.figure()
fig1.suptitle("Turbine Power")
axis1 = fig1.add_subplot(111)
axis1 = performance_results.loc['turbine_power'].plot()
axis1.set_ylabel("Turbine Power [kW] ")
axis1.set_xlabel("Turbine Expansion Rate [ ]")
axis1.axhline(2457, linestyle='--', color='gray', label='given turbine power')
axis1.axvline(2.87, linestyle='--', color='gray')
axis1.legend()
fig1.savefig('images/questao1/ex7_1_a.jpg',
             format='jpeg',
             dpi=200,
             bbox_inches='tight')

performance_results

target = [
    'p06',
    't06',
    'specific_power_turbine_mf',
    'mass_flow_turbine',
    'turbine_power',
    'u_s',
    'specific_thrust',
    'thrust_hot_air',
    'BSFC',
    'gearbox_power',
    'EBSFC',
    'fuel_consumption'
]


values["pr_tl"] = 2.87
turboprop = mf.TurboProp(values)
performance_result = turboprop.sumarise_results()
performance_result = performance_result.append(turboprop.sumarise())
print(performance_result.loc[target].to_markdown())


#### Resultados:
Variamos a taxa de expansão da turbina para encontrar a potência da turbina que corresponde à potência da turbina dada pelo fabricante. A imagem a seguir mostra os resultados obtidos:

<img src="images/questao1/ex7_1_a.jpg" style="width: 600px;"/>

A partir dos resultados acima, encontramos os seguintes parâmetros para calibrar o modelo:

| Parâmetros do Motor               | Valores |
| --------------------------------- | ------- |
| Razão de Pressão na Turbina Livre | 2.87    |
| % de rotação máxima no compressor | 1,00    |


#### Comparação de resultados

| Parâmetro                              | Unidade   | Modelo      | Referência do material | $\Delta$ em % |
| -------------------------------------- | --------- | ----------- | ---------------------- | ------------- |
| Razão de Pressão na Turbina Livre      | -         | 2.87        | 2.87                   | 0.000         |
| Press. Total na saída da Turbina livre | kPa       | 135.944     | 135.98                 | -0.021        |
| Temp. total na saída da Turbina livre  | K         | 931.58      | 931.58                 | 0.000         |
| Trabalho por vazão da Turbina livre    | kJ.s/kg   | 282.358     | 282,83                 | -0.161        |
| Vazão na turbina                       | kg/s      | 8.68121     | 8.69                   | -0.107        |
| Potência da turbina (termodinâmica)    | kW        | 2451.21     | 2456.49                | -0.210        |
| Velocidade de saída do bocal           | m/s       | 385.967     | 385.97                 | -0.002        |
| Empuxo Específico do bocal             | kN.s/kg   | 0.394757    | 0.395                  | -0.067        |
| Empuxo do bocal                        | kN        | 3.35066     | 3.352                  | -0.033        |
| Consumo Específico (BSFC)              | kg/(kN.s) | 7.88615e-05 | 7.87E-05               | 0.205         |
| Potência na caixa de redução           | kW        | 2402.19     | 2407.356               | -0.214        |
| Consumo Específico (EBSFC)             | kg/(kN.s) | 7.88615e-05 | 7.87E-05               | 0.205         |
| Consumo de combustível                 | kg/s      | 0.193306    | 0.193                  | 0.158         |




### Parte 2: Ajuste da Rotação

A potência termodinâmica da turbina é superior à potência limite da caixa de redução de 2050 kW.
Assim, a solução consiste em reduzir a rotação do compressor.

In [ ]:
values = {
    'mass_flow': 8.49,
    't04': 1600,
    'prc': 15.77,
    'u_in': 0,
    'ta': 288.15,
    'pa': 101.30,
    'n_d': 0.85,
    'gamma_d': 1.4,
    'n_c': 0.75,
    'gamma_c': 1.37,
    'n_b': 1,
    'gamma_b': 1.35,
    'n_t': 0.9,
    'gamma_t': 1.33,
    'n_tl': 0.9,
    'gamma_tl': 1.33,
    'n_n': 0.98,
    'gamma_n': 1.36,
    'pc_fuel': 45000,
    'cp_fuel': 1.110,
    'cp_tl': 1.160,
    'cp_tls': 1.160,
    'r': 288.3,
    'pr_tl': 2.87,
    'gearbox_power_ratio': 0.98,
    'propeller_efficiency': 0.85,
}


list_of_n2 = np.linspace(0.9, 1.0, 21)
turboprop = mf.TurboProp(values)
performance_results = turboprop.sumarise_results()
performance_results = performance_results.drop(columns=[1])

for n2 in list_of_n2:
    turboprop.set_n2(n2)
    performance_result = turboprop.sumarise_results()
    performance_results[n2] = performance_result


fig1 = plt.figure()
fig1.suptitle("Turbine Power")
axis1 = fig1.add_subplot(111)
axis1 = performance_results.loc['turbine_power'].plot()
axis1.set_ylabel("Turbine Power [kW] ")
axis1.set_xlabel("Rotation (N2 / N2 @ P.P) [ ]")
axis1.axhline(2050, linestyle='--', color='gray', label='Gearbox Limit Power')
axis1.axvline(0.941, linestyle='--', color='gray')
axis1.legend()
fig1.savefig('images/questao1/ex7_1_b.jpg',
             format='jpeg',
             dpi=200,
             bbox_inches='tight')

performance_results

target = [
    't04',
    'p04',
    'f',
    'p06',
    't06',
    'specific_power_turbine_mf',
    'mass_flow_turbine',
    'turbine_power',
    'u_s',
    'specific_thrust',
    'thrust_hot_air',
    'BSFC',
    'gearbox_power',
    'fuel_consumption'
]


turboprop = mf.TurboProp(values)
turboprop.set_n2(0.941)
performance_result = turboprop.sumarise_results()
performance_result = performance_result.append(turboprop.sumarise())
print(performance_result.loc[target].to_markdown())


A imagem a seguir mostra a variação da potência em função da rotação do motor, para determinar a rotação máxima correspondente ao limite da caixa de redução:

<img src="images/questao1/ex7_1_b.jpg" style="width: 600px;"/>

Parâmetros do motor variados para atingir o desempenho do fabricante:


| Parâmetros do motor               | Valores |
| --------------------------------- | ------- |
| Razão de Pressão na Turbina Livre | 2.87    |
| % de rotação máxima no compressor | 0,941   |

#### Comparação de resultados

| Parâmetro                              | Unidade   | Modelo      | Referência do material | $\Delta$ em % |
| -------------------------------------- | --------- | ----------- | ---------------------- | ------------- |
| % da rotação máxima do compressor      | -         | 0.941       | 0.941                  | 0.0000        |
| Razão de Pressão na Turbina Livre      | -         | 2.87        | 2.87                   | 0.0000        |
| Temp. total na saída da Camara         | K         | 1491.42     | 1491.87                | -0.0301       |
| Press. Total na saída da Camara        | kPa       | 1446.44     | 1447.03                | -0.0407       |
| Razão combustivel/ar                   | -         | 0.0206016   | 0.02061                | -0.0407       |
| Press. Total na saída da Turbina livre | kPa       | 119.654     | 119.68                 | -0.0217       |
| Temp. total na saída da Turbina livre  | K         | 863.739     | 863.96                 | -0.0255       |
| Trabalho por vazão da Turbina livre    | kJ.s/kg   | 264.167     | 264.69                 | -0.1975       |
| Vazão na turbina                       | kg/s      | 7.74105     | 7.75                   | -0.1154       |
| Potência da turbina (termodinâmica)    | kW        | 2044.93     | 2050.13                | -0.2536       |
| Velocidade de saída do bocal           | m/s       | 281.973     | 282.18                 | -0.0733       |
| Empuxo Específico do bocal             | kN.s/kg   | 0.287782    | 0.288                  | -0.0756       |
| Empuxo do bocal                        | kN        | 2.18277     | 2.186                  | -0.1477       |
| Consumo Específico (BSFC)              | kg/(kN.s) | 7.64128e-05 | 7.63e-05               | 0.1478        |
| Potência na caixa de redução           | kW        | 2004.03     | 2009.131               | -0.2538       |
| Consumo de combustível                 | kg/s      | 0.156259    | 0.156                  | 0.1660        |

---

### Exemplo 7.2: Turbohélice - Simulação de Condição de Vôo

#### Descrição do exercício

Simular agora o motor operando a 7.000 m e Mach 0,45, mantendo outros parâmetros de entrada constante em relação ao Exemplo 7.1. Como estratégia de operação considerar N1 =  100% e N2 = 85%.

Parâmetros de funcionamento do motor:

| Condição de funcionamento da linha de base  | Valores   |
| ------------------------------------------- | --------- |
| Fluxo de massa                              | 8,49 kg/s |
| Temperatura de saída na câmara de combustão | 1600 K    |
| Razão de Pressão no Compressor              | 15.77     |


Condição de funcionamento:

| Condição de operação de voo       | Valores   |
| --------------------------------- | --------- |
| Mach                              | 0.45      |
| Pressão atmosférica               | 41,00 kPa |
| Temperatura atmosférica           | 246.55 K  |
| % de rotação máxima do compressor | 0,85      |


Eficiência e dados específicos sobre a relação de calor:

| Componente            | Eficiência Adiabática | $\gamma$                  |
| --------------------- |:---------------------:| ------------------------- |
| entrada               | 0,85                  | 1,40                      |
| compressor            | 0,75                  | 1,37                      |
| combustão             | 1,00                  | 1,35                      |
| turbina do compressor | 0,90                  | 1,33                      |
| turbina livre         | 0,90                  | 1,33                      |
| bocal de ar quente    | 0,98                  | 1,36                      |

Dados termodinâmicos:

| Dados Termodinâmicos                    | Valores           |
| --------------------------------------- | ----------------- |
| PCI                                     | 45000 kJ/kg       |
| Média R                                 | 288,3 m^2/(s^2.K) |
| Cp na câmara de combustão | 1,11 kJ/(kg.K)    |







#### Resolução do exercício

In [ ]:
values = {
    'mass_flow': 8.49,
    't04': 1600,
    'prc': 15.77,
    'mach': 0.45,
    'ta': 246.55,
    'pa': 41,
    'n_d': 0.85,
    'gamma_d': 1.4,
    'n_c': 0.75,
    'gamma_c': 1.37,
    'n_b': 1,
    'gamma_b': 1.35,
    'n_t': 0.9,
    'gamma_t': 1.33,
    'n_tl': 0.9,
    'gamma_tl': 1.33,
    'n_n': 0.98,
    'gamma_n': 1.36,
    'pc_fuel': 45000,
    'cp_fuel': 1.110,
    'cp_tl': 1.160,
    'cp_tls': 1.160,
    'r': 288.3,
    'pr_tl': 2.87,
    'gearbox_power_ratio': 0.98,
    'propeller_efficiency': 0.85,
}

turboprop = mf.TurboProp(values)
turboprop.set_n2(0.85)
performance_results1 = turboprop.sumarise_results()
thermodynamic_results1 = turboprop.sumarise()

performance_results1

target = [
    'p06',
    't06',
    'specific_power_turbine_mf',
    'mass_flow_turbine',
    'turbine_power',
    'u_s',
    'specific_thrust',
    'thrust_hot_air',
    'BSFC',
    'gearbox_power',
    'thrust_propeller',
    'thrust_total',
    'EBSFC',
    'TSFC',
    'fuel_consumption'
]


performance_result = turboprop.sumarise_results()
performance_result = performance_result.append(turboprop.sumarise())
print(performance_result.loc[target].to_markdown())


#### Comparação de resultados

| Parâmetro                              | Unidade   | Modelo      | Referência do material | $\Delta$ em % |
| -------------------------------------- | --------- | ----------- | ---------------------- | ------------- |
| Press. Total na saída da Turbina livre | kPa       | 55.8689     | 55.87                  | -0.0019       |
| Temp. total na saída da Turbina livre  | K         | 798.767     | 798.77                 | -0.0003       |
| Trabalho por vazão da Turbina livre    | kJ.s/kg   | 221.816     | 222.19                 | -0.1683       |
| Vazão na turbina                       | kg/s      | 2.85347     | 2.85                   | 0.1217        |
| Potência da turbina (termodinâmica)    | kW        | 632.945     | 634.00                 | -0.1664       |
| Velocidade de saída do bocal           | m/s       | 366.196     | 366.20                 | -0.0010       |
| Empuxo Específico do bocal             | kN.s/kg   | 0.231       | 0.231                  | 0.0000        |
| Empuxo do bocal                        | kN        | 0.647313    | 0.647                  | 0.0483        |
| Consumo Específico no eixo (BSFC)      | kg/(kW.s) | 8.29033e-05 | 8,28E-05               | 0.1247        |
| Potência na caixa de redução           | kW        | 620.286     | 621.323                | -0.1669       |
| Empuxo produzido pela hélice           | kN        | 3.71415     | 3.720                  | -0.1572       |
| Empuxo total                           | kN        | 4.36146     | 4.368                  | -0.1497       |
| Consumo Específico equivalente(EBSFC)  | kg/(kW.s) | 7.23934e-05 | 7.23e-05               | 0.1291        |
| Consumo Específico empuxo (TSFC)       | kg/(kN.s) | 0.0120311   | 0.012                  | 0.2591        |
| Consumo de combustível                 | kg/s      | 0.0524732   | 0.052                  | 0.9100        |



---

### Exemplo 7.3: TurboFan - Comparando ao Turboélice (Ex. 7.2)

#### Descrição do exercício

Simular um motor turbofan para cumprir a mesma missão do motor turbo hélice do exemplo 7.2, ou seja, gerar o empuxo de 4.26 kN a 7.000 m e a Mach de voo 0,45. Para fazer as simulação tomou-se como referência dados do motor PW545B, com razão de pressão total de 12,5 e razão de passagem 4,2 na condição de decolagem ao nível do mar. Algumas hipóteses foram feitas para simulação como vai ser discutido na sequência.

Parâmetros de funcionamento do motor:

| Parametros de funcionamento do motor        | Valores |
| ------------------------------------------- | ------- |
| Temperatura de saída na câmara de combustão | 1550 K  |
| Razão de Pressão no Compressor              | 8.33    |
| Razão de Pressão em Booster                 | 1.00    |
| Razão de Pressão no fan                     | 1.50    |
| Razao de passagem de ar                     | 4.2     |
| Perda de pressão na câmara de combustão     | 0 %     |

Condição de funcionamento:

| Condição de operação de voo       | Valores   |
| --------------------------------- | --------- |
| mach                              | 0.45      |
| Pressão atmosférica               | 41,00 kPa |
| Temperatura atmosférica           | 246.55 K  |
| % de rotação máxima do compressor | 0,85      |


Eficiência e dados específicos sobre a relação de calor:

| Componente            | Eficiência Adiabática | $\gamma$ |
| --------------------- |:---------------------:| ------------------------- |
| entrada               | 0,97                  | 1,40                      |
| fan                   | 0,90                  | 1,40                      |
| compressor            | 0,75                  | 1,37                      | 
| combustão             | 0,9995                | 1,35                      |
| turbina do compressor | 0,90                  | 1,33                      |
| Turbina do fan        | 0,90                  | 1,33                      |
| Bocal de ar quente    | 0,98                  | 1,36                      |
| Bocal do fan          | 0,98                  | 1,40                      |

Dados termodinâmicos:

| Dados Termodinâmicos                    | Valores           |
| --------------------------------------- | ----------------- |
| PCI                                     | 45000 kJ/kg       |
| Média R                                 | 288,3 m^2/(s^2.K) |
| Cp na câmara de combustão | 1,11 kJ/(kg.K)    |

Os parâmetros do motor variaram para alcançar o exemplo do TurboProp 7.2 de desempenho:

| Parâmetros do motor | Valores    |
| ------------------- | ---------- |
| Fluxo de massa      | 88,42 kg/s |


#### Resolução do Exercício

In [ ]:

values = {
    'pa': 41,
    'ta': 246.55,
    'n_d': 0.97,
    'n_f': 0.90,
    'n_c': 0.75,
    'n_b': 0.9995,
    'n_t': 0.9,
    'n_tf': 0.9,
    'n_n': 0.98,
    'n_nf': 0.98,
    'gamma_d': 1.4,
    'gamma_f': 1.4,
    'gamma_c': 1.37,
    'gamma_b': 1.35,
    'gamma_t': 1.33,
    'gamma_tf': 1.33,
    'gamma_n': 1.36,
    'gamma_nf': 1.40,
    'prc': 8.33,
    'prf': 1.5,
    'bypass_ratio': 4.2,
    'pc_fuel': 45000,
    'cp_fuel': 1.110,
    'r': 288.3,
    't04': 1550,
    'mach': 0.45,
    'mass_flow': 88.42,
}

turboprop = mf.TurboFan(values)
turboprop.set_n2(0.85)
performance_results1 = turboprop.sumarise_results()
thermodynamic_results1 = turboprop.sumarise()

performance_results1

target = [
    'specific_thrust',
    'mass_flow',
    'cold_mass_flow',
    'hot_mass_flow',
    'thrust_total',
    'TSFC',
    'fuel_consumption'
]


performance_result = turboprop.sumarise_results()
performance_result = performance_result.append(turboprop.sumarise())
print(performance_result.loc[target].to_markdown())


#### Comparação de resultados

| Parâmetro                        | Unidade   | Modelo    | Referência do material | $\Delta$ em % |
| -------------------------------- | --------- | --------- | ---------------------- | ------------- |
| Empuxo Específico do bocal       | kN.s/kg   | 0.926056  | 0.8849                 | 4.65092       |
| Vazão massica                    | kg/s      | 29.1712   | 29.18                  | -0.03015      |
| Vazão massica do gerador de gas  | kg/s      | 24.3576   | 24.365                 | -0.03037      |
| Vazão massica do bocal de fan    | kg/s      | 4.81367   | 4.814                  | -0.00685      |
| Empuxo total                     | kN        | 4.45773   | 4.26                   | 4.64154       |
| Consumo Específico empuxo (TSFC) | kg/(kN.s) | 0.020879  | 0.02186                | -4.48764      |
| Consumo de combustível           | kg/s      | 0.0930729 | 0.09313                | -0.06131      |

---

In [ ]:
!jupyter nbconvert --to html --no-input --TemplateExporter.exclude_output_prompt = True gasTurbineReport.ipynb
